In [1]:
import pandas as pd
import requests
import pandas as pd
from tqdm import tqdm
import time
import re
from ast import literal_eval as eval_list


In [2]:
def get_GP_DS_OG(sentences):
    
    queryurl = 'http://ai-capo-api-1:5001/predict/sent'
    header = {"Content-type": "application/json"} 
    
    post_fields_capo2 = {"text": sentence}
    

    response = requests.post(queryurl, json=post_fields_capo2, headers=header)

    if response.status_code==200:
        annotations = response.json()['annotations'] #ast.literal_eval()
    else:
        annotations = ''
    
    return annotations

In [8]:
GP_dev_dataset = pd.read_csv('GP_dev-dataset.csv', sep = '\t', names = ['pmcid', 'text', 'ner'])

In [9]:
GP_dev_dataset

,pmcid,text,ner
0,PMC5962829,Effect of Rebound Exercises and Circuit Traini...,"[[82, 97, 'Type 2 Diabetes', 'DS']]"
1,PMC5962829,Background,NaN
2,PMC5962829,"The incidence of type 2 diabetes mellitus, a c...","[[17, 41, 'type 2 diabetes mellitus', 'DS']]"
3,PMC5962829,Exercise has been documented as being effectiv...,"[[91, 101, 'depression', 'DS'], [120, 133, 'hy..."
4,PMC5962829,"However, there is no consensus regarding the t...","[[194, 209, 'type 2 diabetes', 'DS'], [100, 11..."
...,...,...,...
16103,PMC5723443,"Having understood the practical limitations, s...","[[179, 184, 'tumor', 'DS']]"
16104,PMC5723443,"However, it is important to recognize that eac...",NaN
16105,PMC5723443,"Finally, we believe that our work shows that M...","[[411, 416, 'tumor', 'DS']]"
16106,PMC5723443,We are currently extending this work and are r...,NaN


In [14]:
colNames = ['text', 'ner']
for iter_, row in tqdm(GP_dev_dataset.iterrows(), total = len(GP_dev_dataset)):
    
    sector_dict =[]
    sector_data = pd.DataFrame(columns=colNames)
    
    sentence = row['text']
    
    try:
        ner = eval_list(row['ner'])
    except ValueError:
        ner = ['None']
            
    annotations = get_GP_DS_OG(sentence)
    new_ner =ner+annotations
    


    sector_dict.append(dict(zip(colNames, [sentence,new_ner])))
    sector_CSV = sector_data.append(sector_dict, ignore_index=True)

    sector_CSV.to_csv('CD_GP_DS_OG_dev.csv',encoding='utf-8', index=False,  mode='a', header=False)
    del sector_data # Very important to delete or it will consume the memory


  0%|          | 10/16108 [00:08<3:49:24,  1.17it/s]


KeyboardInterrupt: 

In [3]:
all_dataset = pd.read_csv('CD_GP_DS_OG.csv', header=None, names = ['text', 'ner'])

In [4]:
all_dataset

,text,ner
0,We implemented a two-step approach to detect p...,"['None', [103, 121, 'DS', 'tardive dyskinesia'..."
1,"First, we screened associations by using a gen...","['None', [106, 119, 'DS', 'schizophrenia'], [1..."
2,"Next, we performed a replication analysis in 3...",['None']
3,An association of an SNP in the DPP6 (dipeptid...,"['None', [32, 36, 'GP', 'DPP6'], [38, 73, 'GP'..."
4,The SNP is located in intron-1 of the DPP6 gen...,"['None', [38, 42, 'GP', 'DPP6'], [98, 102, 'GP..."
...,...,...
18235,MSF cultured in differentiation medium with 40...,"[[107, 119, 'triglyceride', 'CD'], [50, 59, 'g..."
18236,Genistein inhibition of adipose differentiatio...,"[[77, 85, 'estrogen', 'CD'], [0, 9, 'Genistein..."
18237,Reduction of ERβ expression by siRNA targeting...,"[[171, 180, 'genistein', 'CD'], [13, 16, 'GP',..."
18238,Conditioned medium from genistein-treated adip...,"[[24, 33, 'genistein', 'CD'], [111, 116, 'OG',..."


In [12]:
import itertools 

def clean_Nones(ner_tags_):
    
    ner_tags = []
    clean_tags = []
        # had to do this as the position of entity tag and entity are exchanged in CD
    for each_ner_tag in ner_tags_:
        if 'CD' in each_ner_tag[3]:
            ner_tags.append([each_ner_tag[0], each_ner_tag[1], each_ner_tag[3], each_ner_tag[2]])
        else:
            ner_tags.append(each_ner_tag)
            

    ner_tags = sorted(ner_tags, key=lambda x: len(x[3]), reverse=True)
    if len(ner_tags)==1 and 'None' in ner_tags:
        return ner_tags
    elif len(ner_tags)>1 and 'None' in ner_tags:
        ner_tags.remove('None')
        return ner_tags
    else:
        return ner_tags
    
    
def remove_overlap(annotation_list):
    
    # first map each item to another item in the list
    mapped_list = list(itertools.combinations(annotation_list, 2))
    unique_maplist = []
    for each_list in mapped_list:
        if each_list[0][2] !=each_list[1][2] and each_list[1][2]== 'CD':
            unique_maplist.append((each_list[0], each_list[1]))

#   now check for the overlaps
    overlapped_tags = []

    for each_map_list in unique_maplist:
        st_1_sp = each_map_list[0][0]
        en_1_sp = each_map_list[0][1]

        st_2_sp = each_map_list[1][0]
        en_2_sp = each_map_list[1][1]

        if st_1_sp <= st_2_sp <= en_1_sp and st_1_sp <= en_2_sp <= en_1_sp:
            if each_map_list[1][3] in each_map_list[0][3]: # process only if the CD is in otherwise not
                overlapped_tags.append(each_map_list[1])
                
#   get the final tags after removing the duplicates

    final_tags = []
    for each_ner_tag in annotation_list:
        if each_ner_tag in overlapped_tags:
            pass
        else:
            final_tags.append(each_ner_tag)
            
    return final_tags

In [13]:
colNames = ['text', 'ner']
for iter_, row in tqdm(all_dataset.iterrows(), total = len(all_dataset)):
    
    sector_dict =[]
    sector_data = pd.DataFrame(columns=colNames)
    
    sentence = row['text']
    
    new_ner = remove_overlap(clean_Nones(eval_list(row['ner'])))
#     annotations = get_GP_DS_OG(sentence)
#     new_ner =ner+annotations
    sector_dict.append(dict(zip(colNames, [sentence,new_ner])))
    sector_CSV = sector_data.append(sector_dict, ignore_index=True)

    sector_CSV.to_csv('all_tags_dataset.csv',encoding='utf-8', index=False,  mode='a', header=False)
    del sector_data # Very important to delete or it will consume the memory


100%|██████████| 18240/18240 [01:18<00:00, 233.06it/s]


In [16]:
file = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/Unsupervised-Protein-Genes-Diseases-Extraction/Scripts/merge_annotations/CD_GP_DS_OG_Dataset_2.py'

In [17]:
file.split('/')[:-1]

['',
 'nfs',
 'gns',
 'literature',
 'Santosh_Tirunagari',
 'GitHub',
 'Unsupervised-Protein-Genes-Diseases-Extraction',
 'Scripts',
 'merge_annotations']